In [1]:
import os
import requests
import pandas as pd

# GDP: Prodotto Interno Lordo (in dollari correnti). Misura la dimensione dell'economia USA ogni trimestre.
# Trimestrale

# GDPC1: PIL corretto per l’inflazione (base chained 2012). Migliore per confronti nel tempo.
# Trimestrale

# QoQ - Quarter over Quarter: percentuale di variazione del PIL da un trimestre al successivo.
# Utile per modelli predittivi e per capire la fase economica.
# Trimestrale

# Consumer Price Index: misura i prezzi al consumo per le famiglie. Inflazione headline.
# Mensile

# Fed Funds Rate: Tasso guida della Federal Reserve. Base di tutti gli altri tassi.
# Mensile

# Treasury 10Y: Interesse sui titoli di Stato a 10 anni. Riflette le aspettative a lungo termine.
# Giornaliero

# Treasury 2Y: Interesse sui titoli di Stato a 2 anni. Sensibile alle politiche della Fed.
# Giornaliero

# Yield Spread: Differenza tra 10Y e 2Y. Se < 0 → segnale di recessione (inversione curva tassi).
# Giornaliero

# Unemployment Rate: percentuale di persone in cerca di lavoro. Indicatore di salute del mercato del lavoro.
# Mensile

# U. Michigan Sentiment: sondaggio mensile: quanto i consumatori sono ottimisti/pessimisti sull’economia.
# Mensile

# 📌 Serie FRED con alias leggibili
series_dict = {
    "GDP": "gdp_nominal",                   # PIL nominale
    "GDPC1": "gdp_real",                    # PIL corretto per inflazione
    "A191RL1Q225SBEA": "gdp_qoq",           # Variazione QoQ del PIL
    "CPIAUCSL": "cpi",                      # CPI headline
    "CPILFESL": "cpi_core",                 # CPI core
    "FEDFUNDS": "ffr",                      # Fed Funds Rate
    "DGS10": "t10y",                        # Treasury 10Y
    "DGS2": "t2y",                          # Treasury 2Y
    "T10Y2Y": "spread_10y_2y",              # Yield Spread 10Y - 2Y
    "UNRATE": "unemployment",               # Tasso di disoccupazione
    "UMCSENT": "sentiment"                  # University of Michigan Sentiment Index
}

api_key = "119c5415679f73cb0da3b62e9c2a534d"
base_url = "https://api.stlouisfed.org/fred/series/observations"

# 📁 Directory di salvataggio
output_dir = "data_macro_new"
os.makedirs(output_dir, exist_ok=True)

# Dizionario per salvare tutti i DataFrame
dataframes = {}

# 🔁 Ciclo per scaricare, pulire e salvare ogni serie
for series_id, alias in series_dict.items():
    print(f"📡 Downloading {alias} ({series_id})...")
    url = f"{base_url}?series_id={series_id}&api_key={api_key}&file_type=json"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        observations = data["observations"]
        df = pd.DataFrame(observations)

        # Pulizia
        df = df[["date", "value"]]
        df["value"] = pd.to_numeric(df["value"], errors="coerce")
        df["date"] = pd.to_datetime(df["date"])
        df = df.sort_values("date").reset_index(drop=True)
        df.rename(columns={"value": f"value_{alias}"}, inplace=True)

        # 💾 Salva CSV nella cartella
        file_path = os.path.join(output_dir, f"{alias}.csv")
        df.to_csv(file_path, index=False)
        print(f"✅ Salvato: {file_path}")

        # Salva per merge finale
        dataframes[alias] = df
    else:
        print(f"❌ Errore {response.status_code} per {alias}")


📡 Downloading gdp_nominal (GDP)...
✅ Salvato: data_macro_new\gdp_nominal.csv
📡 Downloading gdp_real (GDPC1)...
✅ Salvato: data_macro_new\gdp_real.csv
📡 Downloading gdp_qoq (A191RL1Q225SBEA)...
✅ Salvato: data_macro_new\gdp_qoq.csv
📡 Downloading cpi (CPIAUCSL)...
✅ Salvato: data_macro_new\cpi.csv
📡 Downloading cpi_core (CPILFESL)...
✅ Salvato: data_macro_new\cpi_core.csv
📡 Downloading ffr (FEDFUNDS)...
✅ Salvato: data_macro_new\ffr.csv
📡 Downloading t10y (DGS10)...
✅ Salvato: data_macro_new\t10y.csv
📡 Downloading t2y (DGS2)...
✅ Salvato: data_macro_new\t2y.csv
📡 Downloading spread_10y_2y (T10Y2Y)...
✅ Salvato: data_macro_new\spread_10y_2y.csv
📡 Downloading unemployment (UNRATE)...
✅ Salvato: data_macro_new\unemployment.csv
📡 Downloading sentiment (UMCSENT)...
✅ Salvato: data_macro_new\sentiment.csv
